In [1]:
import numpy as np
from numpy.linalg import inv, det
import matplotlib.pyplot as plt
from copy import copy
import random
%matplotlib inline 

## Парето

In [2]:
def find_pareto_optimal(game_matrix):
    optimal_strats = []
    
    def is_optimal(row, col):
        input_item = game_matrix[row, col]
        
        for i in range(game_matrix.shape[0]):
            for j in range(game_matrix.shape[1]):
                cur_item = game_matrix[i, j]

                if (input_item <= cur_item).all() and(input_item < cur_item).any():
                    return False

        return True
    
    for i in range(game_matrix.shape[0]):
        for j in range(game_matrix.shape[1]):
            if is_optimal(i, j):
                optimal_strats.append((i, j))
                
    return optimal_strats            

## Нэш

In [3]:
# массив индексов максимального элемента
def max_item_indexes(vector):
    indexes = np.linspace(0, vector.shape[0] - 1, vector.shape[0], dtype = int)
    
    return indexes[np.isin(vector, np.max(vector))]

def find_nash_optimal(game_matrix):
    # индексы наилучших стратегий первого игрока при i-й стратегии второго
    def first_max_ids(i):
        return max_item_indexes(game_matrix[:, i, 0])
    
    # индексы наилучших стратегий второго игрока при i-й стратегии первого
    def second_max_ids(i):
        return max_item_indexes(game_matrix[i, :, 1])
    
    optimal_strats = []
    
    for i in range(game_matrix.shape[0]):
        second_ids = second_max_ids(i)
        for j in second_ids:
            first_ids = first_max_ids(j)
            
            if i in first_ids:
                optimal_strats.append((i, j))
                    
    return optimal_strats

## Цветная печать

In [4]:
NASH_PREFIX = '\x1b[31m'
PARETO_PREFIX = '\x1b[32m'
BOTH_PREFIX = '\x1b[34m'
POSTFIX = '\x1b[0m'

def color_nash(string):
    return NASH_PREFIX + string + POSTFIX

def color_pareto(string):
    return PARETO_PREFIX + string + POSTFIX

def color_both(string):
    return BOTH_PREFIX + string + POSTFIX

def print_game_solution(game_matrix):
    print(color_nash('Nash optimal'))
    print(color_pareto('Pareto optimal'))
    print(color_both('Both optimal'))
    print()
    
    pareto = find_pareto_optimal(game_matrix)
    nash = find_nash_optimal(game_matrix)
    
    maxlen = 0
    
    for i in range(game_matrix.shape[0]):
        for j in range(game_matrix.shape[1]):
            maxlen = max(maxlen, len(str(game_matrix[i, j])))
    
    for i in range(game_matrix.shape[0]):
        row = []
        for j in range(game_matrix.shape[1]):
            item = (i, j)
            item_str = str(game_matrix[item])
            
            item_str = ' ' * (maxlen - len(item_str)) + item_str
            
            if item in pareto:
                if item in nash:
                    item_str = color_both(item_str)
                else:
                    item_str = color_pareto(item_str)
            elif item in nash:
                item_str = color_nash(item_str)

            row.append(item_str)
            
        print(' '.join(row))

## Условия

In [5]:
epsilon = round(random.randrange(10, 90)/100, 2)

crossroad = np.array(
   [[[1, 1], [1 - epsilon, 2]],
    [[2, 1 - epsilon], [0, 0]]]
)

family_conflict = np.array(
   [[[4, 1], [0, 0]],
    [[0, 0], [1, 4]]]
)

prisoners = np.array(
   [[[ -5, -5], [ 0, -10]],
    [[-10,  0], [-1,  -1]]]
)

var_15 = np.array(
   [[[0, 10], [9, 1]],
    [[7, 8], [6, 11]]]
)

In [6]:
print_game_solution(crossroad)

Nash optimal
Pareto optimal
Both optimal

    [1. 1.] [0.66 2.  ]
[2.   0.66]     [0. 0.]


In [7]:
print_game_solution(family_conflict)

Nash optimal
Pareto optimal
Both optimal

[4 1] [0 0]
[0 0] [1 4]


In [8]:
print_game_solution(prisoners)

Nash optimal
Pareto optimal
Both optimal

  [-5 -5] [  0 -10]
[-10   0]   [-1 -1]


In [9]:
print_game_solution(var_15)

Nash optimal
Pareto optimal
Both optimal

[ 0 10]   [9 1]
  [7 8] [ 6 11]


In [15]:
print_game_solution(np.random.randint(-50, 50, (10, 10, 2)))

Nash optimal
Pareto optimal
Both optimal

[-48  27]   [35 -5] [-50 -33] [ 17 -24] [-20  31]   [34 10] [ 41 -18] [ 11 -46] [-13 -23]   [ 5 42]
  [42 25] [ 10 -15]   [27 -4]   [-9 -5]   [ 2 -9] [ 39 -25] [-36 -34]   [36 34] [-38  23] [-50  -6]
[-43  -6] [-21 -14] [ 17 -39]   [-4  8] [-28 -31] [-20  -5] [-46 -19] [-28  23] [ 23 -43] [-32  23]
[ 39 -21]   [35 12]   [20 11] [-16  28] [ 18 -18] [-16 -24] [-43  39]   [21 21]   [21 42]   [27 25]
[ 20 -23] [-49   1] [-17 -46] [ 26 -14] [-38  38]   [10 38] [ 34 -40] [-22 -13] [-30  30]   [ 4 23]
[-26 -41] [-37 -48] [-14 -46]   [29 -8] [-39  15]   [35 39] [ 20 -27] [-18  19] [  6 -33] [ 28 -11]
  [ 2 35]   [ 3 -4] [ 37 -11]   [34 39] [-27 -28] [-18 -10]   [10 26]   [23 43]   [19 39]   [18 -9]
[ 24 -34]   [35 20] [ -7 -30] [ 38 -36] [ 40 -18]   [ 1 15] [ 46 -18] [ 22 -13] [ 10 -33] [ 43 -47]
  [24  5] [-50 -47] [-26 -14] [ -6 -49] [-41  22] [-23 -34] [ 28 -11] [  0 -33]   [45 19] [-42  14]
[-42 -17]   [22 29] [-47  36] [-38  48]   [22 49]   [29 -6